## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
from IPython.display import clear_output
import time
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.drop("Unnamed: 0", axis=1, inplace=True)
city_data_df.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Weather_Description
0,cape town,-33.93,18.42,64.00,93,0,9.17,ZA,2020-10-23 02:42:02,clear sky
1,kant,27.80,79.80,77.31,36,0,4.54,IN,2020-10-23 02:45:11,clear sky
2,pacific grove,36.62,-121.92,61.00,82,1,3.36,US,2020-10-23 02:45:11,clear sky
3,hilo,19.73,-155.09,84.20,66,1,14.99,US,2020-10-23 02:42:17,clear sky
4,tura,25.52,90.22,75.60,88,47,1.66,IN,2020-10-23 02:45:12,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
def temperature_input():
    try:
        min_temp, max_tep = input("Please enter your prefered minumum temperature followed by your prefered maximum temperature seperated by a ',':\n ").strip().split(",")
    except ValueError:
        clear_output()
        print("\nPlease Enter 2 integers seperated by a ','")
        time.sleep(3)
        clear_output()
        min_temp, max_temp = temperature_input()
    return min_temp, max_tep


min_temp, max_temp = temperature_input()
   

Please enter your prefered minumum temperature followed by your prefered maximum temperature seperated by a ',':
 25, 70


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
Max_Min_Temp_df=city_data_df.loc[(city_data_df.Max_Temp >= int(min_temp)) & (city_data_df.Max_Temp <= int(max_temp))]

In [5]:
# 4a. Determine if there are any empty rows.
Max_Min_Temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1025 entries, 0 to 1840
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 1025 non-null   object 
 1   Lat                  1025 non-null   float64
 2   Lng                  1025 non-null   float64
 3   Max_Temp             1025 non-null   float64
 4   Humidity             1025 non-null   int64  
 5   Cloudiness           1025 non-null   int64  
 6   Wind_Speed           1025 non-null   float64
 7   Country              1018 non-null   object 
 8   Date                 1025 non-null   object 
 9   Weather_Description  1025 non-null   object 
dtypes: float64(4), int64(2), object(4)
memory usage: 88.1+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
Max_Min_Temp_df=Max_Min_Temp_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = Max_Min_Temp_df[["City", "Country", "Max_Temp", "Weather_Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max_Temp,Weather_Description,Lat,Lng,Hotel Name
0,cape town,ZA,64.00,clear sky,-33.93,18.42,
2,pacific grove,US,61.00,clear sky,36.62,-121.92,
12,port alfred,ZA,68.09,overcast clouds,-33.59,26.89,
18,thompson,CA,26.60,light snow,55.74,-97.86,
19,east london,ZA,67.42,broken clouds,-33.02,27.91,
22,xam nua,LA,69.17,overcast clouds,20.42,104.04,
23,port elizabeth,ZA,66.20,mist,-33.92,25.57,
24,ushuaia,AR,42.80,scattered clouds,-54.80,-68.30,
25,barrow,US,33.80,mist,71.29,-156.79,
26,port alfred,ZA,68.09,overcast clouds,-33.59,26.89,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
counter=0
index_2=0
# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():  
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #Add lat and lng to params dictionary
    params["location"] = f"{lat},{lng}"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    response=requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        print(f'Success {index_2} of {len(hotel_df)} | {"{:.2f}".format((index_2/len(hotel_df)*100))}%')
        counter+=1
        index_2+=1
        if counter == 10:
            clear_output()
            counter=0
        
    except IndexError:
        print(f'\nFaliure {index_2} of {len(hotel_df)} | {"{:.2f}".format((index_2/len(hotel_df.index)*100))}% \n\t\tSkipping...\n')
        counter+=1
        index_2+=1
        if counter == 10:
            clear_output()
            counter=0
        pass

Success 1010 of 1018 | 99.21%
Success 1011 of 1018 | 99.31%
Success 1012 of 1018 | 99.41%
Success 1013 of 1018 | 99.51%
Success 1014 of 1018 | 99.61%
Success 1015 of 1018 | 99.71%
Success 1016 of 1018 | 99.80%

Faliure 1017 of 1018 | 99.90% 
		Skipping...



In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""
#Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

KeyError: 'column1'

In [ ]:
# 11a. Add a marker layer for each city to the map. 
hotel_info = """
<dl>
<dt>Hotel Name</dt><dd>{Country}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max_Temp</dt><dd>{Max_Temp} °F</dd>
</dl>
"""
# 11b. Display the figure
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max_Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

In [ ]:
clean_hotel_df.head()